In [1]:
from flask import Flask, render_template, request, jsonify,url_for
import openai
import pandas as pd
import os
from langchain.agents.agent_types import AgentType
from langchain.agents import load_tools
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_community.chat_models import ChatOpenAI
from langchain.agents.agent_types import *
from langchain_openai import OpenAI
import regex as re
import base64
import shutil
import time

app = Flask(__name__)
openai.api_key = os.environ.get('OPENAI_API_KEY')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process', methods=['POST'])
def process():
    start_time=time.time()
    question = request.form.get('question')
    data_source = request.form.get('data_source')
    # print(question)
    print(data_source)
    
    df_BLK = pd.read_csv('C:\CODE\Demo\BLk_DATASET.csv')
    df_blk_meta=pd.read_csv('C:\CODE\Demo\df_blk_meta.csv')
    df_dtc_data=pd.read_csv('C:\CODE\Demo\dtc_dataset.csv')
    
    agent_blk_meta = create_pandas_dataframe_agent(OpenAI(temperature=0), df_blk_meta, verbose=True)

    agent_blk_meta = create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_blk_meta,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )

    agent_blk = create_pandas_dataframe_agent(OpenAI(temperature=0), df_BLK, verbose=True)

    agent_blk = create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_BLK,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    
    agent_dtc_data= create_pandas_dataframe_agent(OpenAI(temperature=0), df_dtc_data, verbose=True)

    agent_dtc_data= create_pandas_dataframe_agent(
        ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        df_dtc_data,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    
    if data_source == 'metadata':
 
        df_a=agent_blk_meta.run(question)
        print(question)
        # print(df_a)

    elif data_source == 'blk':
        
        df_a=agent_blk.run(question)
        print(question)
    elif data_source == 'dtc':
        
        df_a=agent_dtc_data.run(question)

    else:
        print("Please go back and select a source from the drop down")
    
    print("#########df_a#####\n",df_a)
    
    match = re.search(r'\b[\w-]+\.(?:png|jpg|jpeg|gif)\b', df_a)
    print("here is the match if found:",match)
    
    image_url=None
    if match:
        filename = match.group()
        source_file = r'C:\CODE\Demo\static\{}'.format(filename)  # Using the filename variable
        shutil.move(r'C:\CODE\Demo\{}'.format(filename),r'C:\CODE\Demo\static\collectionimagess')
        print('**********************************')
        print(source_file)
        image_url =url_for('static',filename= r'collectionimagess/'+filename)
    print(image_url)
    end_time=time.time()
    elapsed_time = end_time - start_time
    print(f"Time consumed for processing request: {elapsed_time} seconds")
    

    return jsonify({'response': df_a,'image_path':image_url})

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/May/2024 11:01:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 11:01:31] "GET /static/logo.jpg HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2024 11:01:31] "GET /favicon.ico HTTP/1.1" 404 -


blk


c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['production_date'] = pd.to_datetime(df['production_date'])\n\nyear_2022_cars = df[df['production_date'].dt.year == 2022]['model_series']\n\nyear_2022_cars"}`


NameError: name 'pd' is not defined
Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\ndf['production_date'] = pd.to_datetime(df['production_date'])\n\nyear_2022_cars = df[df['production_date'].dt.year == 2022]['model_series']\n\nyear_2022_cars"}`
responded: It seems that the `pd` module is not imported. Let me import it and try again.

1       S-Class 223
3           GLC 253
4           GLC 253
5       C-Class 206
11      C-Class 206
           ...     
9995        CLA 118
9996        GLC 253
9997    E-Class 213
9998        GLE 167
9999    A-Class 177
Name: model_series, Length: 8260, dtype: object
Invoking: `python_repl_ast` with `{'query': "import matplotlib.pyplot as plt\n\nyear_2022_cars.value_counts().plot(kind='bar')\

...

127.0.0.1 - - [14/May/2024 11:18:39] "POST /process HTTP/1.1" 200 -


I apologize, but it seems that I don't have access to the `data.csv` file. Please make sure that the file is in the correct location and that I have permission to access it. Once I have access to the data, I will be able to calculate the average of the parking days for GLC and generate the plot in PNG format.

> Finished chain.
what is average of the parking days of glc? give me in png format?
#########df_a#####
 I apologize, but it seems that I don't have access to the `data.csv` file. Please make sure that the file is in the correct location and that I have permission to access it. Once I have access to the data, I will be able to calculate the average of the parking days for GLC and generate the plot in PNG format.
here is the match if found: None
None
Time consumed for processing request: 16.317025661468506 seconds
blk


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['model_series'] == 'GLC 253']['parking_days'].mean()"}`


354.75504484304

127.0.0.1 - - [14/May/2024 11:19:00] "POST /process HTTP/1.1" 200 -


The average number of parking days for the GLC model is approximately 354.76 days.

> Finished chain.
what is average of the parking days of glc?
#########df_a#####
 The average number of parking days for the GLC model is approximately 354.76 days.
here is the match if found: None
None
Time consumed for processing request: 14.726916790008545 seconds
blk


> Entering new AgentExecutor chain...


[2024-05-14 11:20:28,901] ERROR in app: Exception on /process [POST]
Traceback (most recent call last):
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\site-packages\langchain\agents\output_parsers\openai_functions.py", line 48, in _parse_ai_message
    _tool_input = json.loads(function_call["arguments"], strict=False)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\json\__init__.py", line 359, in loads
    return cls(**kw).decode(s)
           ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ansdwiv\AppData\Local\miniconda3\Lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During han